## Environmenal Sustainability Echo 'Not a Chat GPT' 
## Quick rough example for every one to try out Echo of domain data
## dowload your domain and try the Echo 

### This is a simple Echo which loops to ask questions
### select the question by the row number
### repeat
### the close button is probamatic in Mac OS thread, not working

In [1]:
import json
import random
import pandas as pd
import tkinter as tk
from tkinter import simpledialog
import threading

In [ ]:
def load_data():
    try:
        with open('train.json', 'r') as file:
            data = json.load(file)
            if isinstance(data, list):  
                return data
            else:
                raise ValueError("Loaded data is not a list.")
    except Exception as e:
        print(f"Error loading data: {e}")
        return []  
    
def get_random_questions(data, num_questions=15):  
    return random.sample(data, min(num_questions, len(data)))

class gen_chat_OneShotPrompt:
    def __init__(self, data):
        self.data = data

    def generate_prompt(self, question, index=0):
        return f"Q: {question}\nA: {self.data[index]['chunk']}"

class gen_chat_FewShotPrompt:
    def __init__(self, data):
        self.data = data

    def generate_prompt(self, question, examples=2):
        prompt = f"Q: {question}\nA: {self.data[0]['chunk']}\n\n"
        for i in range(1, examples):
            prompt += f"Q: {self.data[i]['question']}\nA: {self.data[i]['chunk']}\n\n"
        return prompt

class gen_chat_ConversationalPrompt:
    def __init__(self, data):
        self.data = data

    def generate_prompt(self, question, index=0):
        conversation = [
            {"role": "system", "content": "You are an environmental sustainability expert."},
            {"role": "user", "content": question},
            {"role": "assistant", "content": self.data[index]['chunk']}
        ]
        return conversation
    
def select_question(data, root):  
    if not data:  
        print("No questions available.")
        return None

    random_questions = random.sample(data, min(15, len(data)))  
    question_list = "\n".join([f"{i}: {question['question']}" for i, question in enumerate(random_questions)])
    
    selected_row = simpledialog.askstring("Select a Question", f"Choose a question by entering the row number:\n\n{question_list}", parent=root)
    
    if selected_row and selected_row.isdigit():
        selected_index = int(selected_row)
        if 0 <= selected_index < len(random_questions):
            return data.index(random_questions[selected_index]) 
    return None 

def show_popup(content, root, on_ask_another, on_close):
    if hasattr(show_popup, 'current_popup') and show_popup.current_popup is not None:
        show_popup.current_popup.destroy()

    popup = tk.Toplevel(root)
    popup.title("Generated Prompts")
    show_popup.current_popup = popup

    short_content = content[:500] + '...' if len(content) > 500 else content

    label = tk.Label(popup, text=short_content, justify='left')
    label.pack(side="top", fill="both", padx=10, pady=10)

    ask_another_button = tk.Button(popup, text="Ask Another Question", command=lambda: on_ask_another(popup))
    ask_another_button.pack()

    close_button = tk.Button(popup, text="Close", command=lambda: on_close(popup))
    close_button.pack()

def close_program(popup=None):
    if popup:
        popup.destroy()  
    print("Closing the program.")
    root.quit() 
    
    
def gen_ai_POC(data):  
    def ask_another_question(popup=None):
        if popup:
            popup.destroy()

        selected_index = select_question(data, root)  
        if selected_index is not None:
            question = data[selected_index]['question']
            print(f"Selected question (row {selected_index}): {question}")
            
            one_shot_prompt = f"One-shot Prompt for: {question}"
            few_shot_prompt = f"Few-shot Prompt for: {question}"
            conversation = f"Conversational Prompt for: {question}"

            final_output = (f"Selected Question (row {selected_index}): {question}\n\n"
                            f"One-shot Prompt:\n{one_shot_prompt}\n\n"
                            f"Few-shot Prompt:\n{few_shot_prompt}\n\n"
                            f"Conversational Prompt:\n{conversation}")

            show_popup(final_output, root, ask_another_question, close_program)
        else:
            print("No valid question selected.")

    return ask_another_question 

def run_tkinter(data):
    global root
    root = tk.Tk()
    root.withdraw()
    ask_another_question = gen_ai_POC(data)  
    ask_another_question()  
    root.mainloop()
    
if __name__ == "__main__":
    data = load_data()  
    run_tkinter(data)    

Selected question (row 2398): Who is responsible for collecting energy efficiency data for reporting?
Selected question (row 1127): Can you tell me about the durability features of the latest iPhones?
